# Introduction to In-context learning
1. **🤖 What is in-context learning (ICL)**
2. **🦮 Zero-shot vs. Few-shot ICL**
3. **🎨 Prompt design**
4. **✋ Hands-on: Transforming a dataset into a few-shot prompt-label dataset and evaluating existing models**


In [1]:
import sys
!{sys.executable} -m pip install git+https://github.com/fewshot-goes-multilingual/promptsource

  Cloning https://github.com/fewshot-goes-multilingual/promptsource to /tmp/pip-req-build-9supn9vs
  Running command git clone --filter=blob:none --quiet https://github.com/fewshot-goes-multilingual/promptsource /tmp/pip-req-build-9supn9vs
  Resolved https://github.com/fewshot-goes-multilingual/promptsource to commit eb6d175b2c397cb7ee2aa46c334c17e3238a49cc
  Preparing metadata (setup.py) ... done


# 1. 🤖 In-context learning (ICL)

In context learning is a use of a generative model, where the description of a desired task is a part of the input. 

While pre-training, the model is trained on an objective of "guessing" the right word in context. This is achieved by objectives like Masked Language Modeling (MLM) or Causal Language Modeling (CLM). During these objectives, the model aquires an inherent understanding of the language. 

After pre-training, traditionaly, we would then fine-tune the model through Supervised ML for a specific task for which we need:
* Training data (input and label pairs)
* Adding a specific layer ("head") to the model relevant to our desired task
The resulting model is fit for that one specific task.

When we talk about In-context learning, we mean a property of a model to solve tasks it was not fine-tuned for with only instructions provided in the user's input, aka a prompt. In-context learning is a property usually seen in Large Language Models


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
 
model_path = "gaussalgo/mt5-base-priming-QA_en-cs"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


/home/nikola/L2L_MLPrague23/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
prompt = """
What is meant by: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. 
I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial"
I really had to see this for myself. The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life.
In particular she wants to focus her attentions to making some sort of documentary on what the
average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. 
"""

inputs = tokenizer([prompt], return_tensors="pt", padding=True)
outputs = model.generate(**inputs.to(model.device))
outputs_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# predictions:
outputs_str

['The plot is centered around a young Swedish drama student named Lena']

## 2. 🦮 Zero-shot vs few-shot in-context learning
The model might understand the task from the input, but it does not know how do we expect it to respond. Therefore, if we have the model adjusted for such use, we can show it the format of the task from a few input-output examples and see if it comprehends.

This approach is called in-context few-shot learning: In addition to the description of the task, we give the model a few input-output examples (demonstrations). Given these, the model has much easier time understanding the format of the interaction that we expect from it. The demonstration are the only lead the model has to understand the task at hand. We can see, that if we pick only examples with a negative sentiment, the model is unable to pick the correct label. 

In this setting, we need to standardize the format of prediction, so that the model can rely on it.


In [4]:
input_zero_shot = """
Question: What is the sentiment of the context: positive or negative? 
Context: I am very happy to be here today.
Answer:""
"""
input_few_shot_not_heterogenic = """
Question: What is the sentiment of the context: positive or negative? 
Context: He said, that the consert was very dull.
Answer:"negative"
Question: What is the sentiment of the context: positive or negative? 
Context: She came from school sad and lonely.
Answer:"negative"
Question: What is the sentiment of the context: positive or negative? 
Context: I am very happy to be here today.
Answer:""
"""
input_few_shot = """
Question: What is the sentiment of the context: positive or negative? 
Context: He said, that the consert was very dull.
Answer:"negative"
Question: What is the sentiment of the context: positive or negative? 
Context: She came from school smiling and singing.
Answer:"positive"
Question: What is the sentiment of the context: positive or negative? 
Context: I am very happy to be here today.
Answer:""
"""
inputs = tokenizer([input_zero_shot,input_few_shot_not_heterogenic,  input_few_shot], return_tensors="pt", padding=True)
outputs = model.generate(**inputs.to(model.device))
outputs_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# predictions:
outputs_str

['happy', 'positive or negative', 'positive']

## 3. 🎨 What should the prompts look like?
For training a custom in-context learner we need text pairs of a prompt and label. While in the above example we see a unified prompt, in training it is beneficial to create multiple prompts for one task, as we want to support the models capability to understand the task by its description, not by identifying a task by a template. This diversivication should yield a benefit of having the model understanding never before seen tasks better.

In [5]:
#TODO write your own prompt (format string)

In [12]:
from datasets import load_dataset
from promptsource.templates import DatasetTemplates

dataset = load_dataset('super_glue', 'boolq', split="validation[:10%]")

prompts = DatasetTemplates("super_glue/boolq")
print(prompts.all_template_names)
prompt = prompts['GPT-3 Style']


Found cached dataset super_glue (/home/nikola/.cache/huggingface/datasets/super_glue/boolq/1.0.3/bb9675f958ebfee0d5d6dc5476fafe38c79123727a7258d515c450873dbdbbed)


['GPT-3 Style', 'I wonder…', 'after_reading', 'based on the following passage', 'based on the previous passage', 'could you tell me…', 'exam', 'exercise', 'valid_binary', 'yes_no_question']


### 3.1 Evaluation
Let's evaluate our model on a dataset created using the promptsource library and a dataset about if the answer to a question is in the context. (The model was not trained on this dataset)

In [11]:
from tqdm import tqdm

predictions = []
references = [x==1 for x in dataset["label"]]

# Get predictions
for item in tqdm(dataset):
    model_input_string = prompt.apply(item)
    inputs = tokenizer(model_input_string,padding=True, truncation=True, return_tensors="pt")
    outputs = model.generate(**inputs.to(model.device))
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    predictions.append(response_text)


  0%|          | 0/327 [00:00<?, ?it/s]

["EXAM\n1. Answer by yes or no.\n\nDocument: Ethanol fuel -- All biomass goes through at least some of these steps: it needs to be grown, collected, dried, fermented, distilled, and burned. All of these steps require resources and an infrastructure. The total amount of energy input into the process compared to the energy released by burning the resulting ethanol fuel is known as the energy balance (or ``energy returned on energy invested''). Figures compiled in a 2007 report by National Geographic Magazine point to modest results for corn ethanol produced in the US: one unit of fossil-fuel energy is required to create 1.3 energy units from the resulting ethanol. The energy balance for sugarcane ethanol produced in Brazil is more favorable, with one unit of fossil-fuel energy required to create 8 from the ethanol. Energy balance estimates are not easily produced, thus numerous such reports have been generated that are contradictory. For instance, a separate survey reports that productio

  0%|          | 1/327 [00:00<05:03,  1.08it/s]

["EXAM\n1. Answer by yes or no.\n\nDocument: Property tax -- Property tax or 'house tax' is a local tax on buildings, along with appurtenant land. It is and imposed on the Possessor (not the custodian of property as per 1978, 44th amendment of constitution). It resembles the US-type wealth tax and differs from the excise-type UK rate. The tax power is vested in the states and is delegated to local bodies, specifying the valuation method, rate band, and collection procedures. The tax base is the annual rental value (ARV) or area-based rating. Owner-occupied and other properties not producing rent are assessed on cost and then converted into ARV by applying a percentage of cost, usually four percent. Vacant land is generally exempt. Central government properties are exempt. Instead a 'service charge' is permissible under executive order. Properties of foreign missions also enjoy tax exemption without requiring reciprocity. The tax is usually accompanied by service taxes, e.g., water tax,

  1%|          | 2/327 [00:02<06:26,  1.19s/it]

...

["EXAM\n1. Answer by yes or no.\n\nDocument: Post-office box -- Street Addressing will have the same street address of the post office, plus a ``unit number'' that matches the P.O. Box number. As an example, in El Centro, California, the post office is located at 1598 Main Street. Therefore, for P.O. Box 9975 (fictitious), the Street Addressing would be: 1598 Main Street Unit 9975, El Centro, CA. Nationally, the first five digits of the zip code may or may not be the same as the P.O. Box address, and the last four digits (Zip + 4) are virtually always different. Except for a few of the largest post offices in the U.S., the 'Street Addressing' (not the P.O. Box address) nine digit Zip + 4 is the same for all boxes at a given location.\nQuestion: does p o box come before street address?", 'No']


  6%|▋         | 21/327 [00:17<03:49,  1.33it/s]

["EXAM\n1. Answer by yes or no.\n\nDocument: Spark plug -- A spark plug (sometimes, in British English, a sparking plug, and, colloquially, a plug) is a device for delivering electric current from an ignition system to the combustion chamber of a spark-ignition engine to ignite the compressed fuel/air mixture by an electric spark, while containing combustion pressure within the engine. A spark plug has a metal threaded shell, electrically isolated from a central electrode by a porcelain insulator. The central electrode, which may contain a resistor, is connected by a heavily insulated wire to the output terminal of an ignition coil or magneto. The spark plug's metal shell is screwed into the engine's cylinder head and thus electrically grounded. The central electrode protrudes through the porcelain insulator into the combustion chamber, forming one or more spark gaps between the inner end of the central electrode and usually one or more protuberances or structures attached to the inner

  6%|▋         | 21/327 [00:19<04:38,  1.10it/s]


KeyboardInterrupt: 

In [10]:
# Accuracy
correct_predictions = sum([pred == str(true) for pred, true in zip(predictions, references)])
incorrect_predictions = sum([pred != str(true) for pred, true in zip(predictions, references)])

accuracy = correct_predictions / (correct_predictions+incorrect_predictions)
print("Prediction using '%s' classifier; accuracy: %s" % (model.config.model_type, accuracy))  

Prediction using 'mt5' classifier; accuracy: 0.0


# 4. ✋ Hands on: Creation of an evaluation dataset 

* Download an existing dataset and transform it into a prompt input - label pair (either by creating your own prompt or by using the promtsource library).
  * Text classification (https://huggingface.co/datasets/imdb)
  * Named Entity Recognition (https://huggingface.co/datasets/polyglot_ner/viewer/en/train)
  * Question Answering (https://huggingface.co/datasets/squad_v2)
  * or other

* Adjust the evaluation script accordingly

* Create a function which will generate a few-shot (the prompt will include few demonstrations of the same task) prompt and label pairs.

* evaluate some existing ICL models on your dataset (try both zero-shot and few-shot prompts):

  * https://huggingface.co/google/flan-t5-large
  * https://huggingface.co/allenai/mtk-instruct-3b-def-pos

In [9]:
def pick_random_demonstrations():
    # From your custom dataset pick random demostrations (prompt-label pairs)
    pass

def create_few_shot_prompt():
    # With the pick_random_demonstrations() function create a new prompt
    pass

# Get models predictions

# Evaluate (depending on your dataset you may need to change the evaluation script) 